In [10]:
import random, os, pickle, time
import pandas as pd
import numpy as np
import networkx as nx

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

from collections import defaultdict


# Set environment variables for reproducibility and safety
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import precision_score, recall_score, accuracy_score

# 1. Configuration & Seeding
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

In [11]:
name = 'book'

In [ ]:
class TCKG:
    def __init__(self, tckg_csv_path):
        self.adj_list = defaultdict(list)
        
        print(f"Loading TCKG from {tckg_csv_path}...")
        df_tckg = pd.read_csv(tckg_csv_path, usecols=['head_id', 'relation_id', 'tail_id'])
        
        offset = df_tckg['relation_id'].max()
            
        # self.num_relations = offset * 2 + 1# Total relation_id (bidirection)
        # print(f"Inverse Relation Offset: {offset}. Total Relation Space: {self.num_relations}")

        data = df_tckg[['head_id', 'relation_id', 'tail_id']].to_numpy() # Using numpy to speedup
        
        for h, r, t in data:
            h, r, t = int(h), int(r), int(t)

            self.adj_list[h].append((r, t))
            self.adj_list[t].append((r + offset, h)) # Ex: r=5 (watched) -> r_inv=105 (watched_by)

        print(f"TCKG Loaded successfully. Graph construction complete.")

    def get_neighbors(self, node_id):
        return self.adj_list[node_id]

    def get_all_nodes(self):
        return list(self.adj_list.keys())

In [13]:
class TimeAwareRewardFunction(nn.Module):
    def __init__(self, user_embs, entity_embs, relation_embs, interaction_cluster_ids, bias_embs=None, temperature= None):
        """
        Args:
            user_embs (nn.Embedding): Embedding của User (e_u)
            entity_embs (nn.Embedding): Embedding của Item/Entity (e_v)
            relation_embs (nn.Embedding): Embedding của Relation (dùng để lấy V_U)
            interaction_cluster_ids (list or tensor): Danh sách các Relation ID đại diện cho Time Clusters.
                                                      Ví dụ: [20, 21, 22] ứng với interacted_0, interacted_1...
                                                      Đây chính là tập {V_U^1, ..., V_U^L}
            bias_embs (nn.Embedding, optional): Bias của entity (b_v). Nếu None sẽ tự khởi tạo.
        """
        super(TimeAwareRewardFunction, self).__init__()
        
        self.user_embs = user_embs
        self.entity_embs = entity_embs
        self.relation_embs = relation_embs
        
        # Danh sách ID của các cluster tương tác theo thời gian (V_U)
        # Chuyển thành tensor để tính toán song song
        self.register_buffer('cluster_ids', torch.tensor(interaction_cluster_ids, dtype=torch.long))
        
        # Entity Bias (b_v) - Eq (11)
        if bias_embs is None:
            num_entities = entity_embs.num_embeddings
            self.bias_embs = nn.Embedding(num_entities, 1)
            nn.init.zeros_(self.bias_embs.weight) # Khởi tạo bias bằng 0
        else:
            self.bias_embs = bias_embs

        if temperature is None:
            self.temperature = self.entity_embs.embedding_dim ** 0.5
        else:
            self.temperature = temperature

    def calculate_weights(self, history_relation_ids):
        """
        Thực hiện Eq (13): Tính trọng số w_h dựa trên tần suất xuất hiện trong lịch sử.
        
        Args:
            history_relation_ids: (Batch, Max_History_Len) - Chứa relation ID trong quá khứ của user.
            
        Returns:
            weights: (Batch, Num_Clusters) - Vector W_hu
        """
        # 1. So khớp History với Cluster IDs
        # history: (B, H, 1)
        # clusters: (1, 1, L)
        # matches: (B, H, L) -> True nếu relation tại history khớp với cluster ID
        hist_expanded = history_relation_ids.unsqueeze(-1)
        clusters_expanded = self.cluster_ids.view(1, 1, -1)
        
        matches = (hist_expanded == clusters_expanded).float()
        
        # 2. Đếm số lần xuất hiện (Tử số Eq 13)
        # Sum theo chiều History (dim=1) -> (B, L)
        counts = matches.sum(dim=1) 
        
        # 3. Tính độ dài q thực tế (Mẫu số Eq 13)
        # q = tổng số lần xuất hiện của bất kỳ cluster nào trong history (tránh tính padding 0)
        q = counts.sum(dim=1, keepdim=True)
        
        # 4. Normalize để ra trọng số (tránh chia cho 0)
        weights = counts / (q + 1e-9)
        
        return weights

    def forward(self, user_ids, item_ids, history_relation_ids):
        """
        Tính Reward Score g_R(v | u)
        
        Args:
            user_ids: (Batch,)
            item_ids: (Batch,) - Item đích (v_hat) mà Agent dự đoán/dừng lại
            history_relation_ids: (Batch, History_Len) - Lịch sử relation của user
            
        Returns:
            scores: (Batch,) - Điểm reward
        """
        # --- BƯỚC 1: Lấy Embeddings cơ bản ---
        u_e = self.user_embs(user_ids)       # (B, Dim) -> e_u
        v_e = self.entity_embs(item_ids)     # (B, Dim) -> e_v
        v_b = self.bias_embs(item_ids).squeeze(-1) # (B,) -> b_v
        
        # --- BƯỚC 2: Tính Personalized Interaction Relation (Eq 12) ---
        # r_vu^T = W_hu * V_U
        
        # a. Tính weights (B, L)
        weights = self.calculate_weights(history_relation_ids)
        
        # b. Lấy embedding của các cluster V_U^1...L
        # cluster_embs shape: (L, Dim)
        cluster_embs = self.relation_embs(self.cluster_ids)
        
        # c. Tính tổng có trọng số
        # (B, L) x (L, Dim) -> (B, Dim)
        r_interaction = torch.matmul(weights, cluster_embs)
        
        # --- BƯỚC 3: Tính Score (Eq 11 & Final Eq) ---
        # g = (e_u + r_interaction) . e_v + b_v
        
        # Dot product: (e_u + r) * e_v
        query_vector = u_e + r_interaction # (B, Dim)
        dot_product = torch.sum(query_vector * v_e, dim=1) # (B,)
        
        scores = dot_product + v_b # Cộng bias
        
        # 1. Scale Score (Chia cho nhiệt độ)
        scaled_score = scores / self.temperature
        
        # 2. Áp dụng Sigmoid
        rewards = torch.sigmoid(scaled_score)
        
        return rewards


In [14]:
class TPRecEnvironment(nn.Module):
    def __init__(self, kg, entity_embeddings, relation_embeddings, reward_function, max_path_len=3, history_len=3):
        """
        Thêm tham số: reward_function
        """
        super(TPRecEnvironment, self).__init__()
        self.kg = kg
        self.entity_embs = entity_embeddings
        self.relation_embs = relation_embeddings
        self.max_path_len = max_path_len
        self.history_len = history_len
        
        # LƯU REWARD FUNCTION ĐƯỢC TRUYỀN VÀO
        self.reward_function = reward_function 

        # State tracking
        self.current_entities = None
        self.current_users = None
        self.path_history = None
        self.step_counter = 0

    def reset(self, user_ids):
        """
        Initiate status s_0 = (u, u, ∅)
        """
        batch_size = user_ids.size(0)

        self.current_users = user_ids       # Multiple values because of batch_size
        self.current_entities = user_ids    # Initiate status s_0 = (u, u, ∅)
        
        # History h_k: store (relation, entity)
        # Bổ sung device=user_ids.device để tránh lỗi khi dùng GPU
        self.path_history = torch.zeros((batch_size, self.history_len * 2), 
                                        dtype=torch.long, 
                                        device=user_ids.device)

        self.step_counter = 0        
        return self._get_state_embedding()

    def _get_state_embedding(self):
        """
        State = [User_Emb, Flattened_History_Emb, Current_Entity_Emb]
        Input cho Policy Network sẽ là vector nối dài của 3 thành phần này.
        """
        u_emb = self.entity_embs(self.current_users)
        e_emb = self.entity_embs(self.current_entities)
        
        # 2a: Tách index của Relation và Entity bằng kỹ thuật Slicing
        r_indices = self.path_history[:, 0::2] # (Batch, history_len)
        e_indices = self.path_history[:, 1::2] # (Batch, history_len)
        
        # 2b: Lookup Embedding
        r_vecs = self.relation_embs(r_indices) 
        e_vecs = self.entity_embs(e_indices)
        
        # 2c: Kết hợp Relation và Entity tại mỗi bước
        step_vecs = torch.cat([r_vecs, e_vecs], dim=2)
        
        # 2d: Làm phẳng (Flatten) toàn bộ lịch sử
        batch_size = step_vecs.size(0)
        h_emb_flat = step_vecs.view(batch_size, -1)

        # 3. KẾT HỢP TẤT CẢ (Concatenate)
        # Kết quả có cấu trúc logic tương đương: [User_Emb, r1, e1, r2, e2, r3, e3, Current_Entity_Emb]
        state_vector = torch.cat([u_emb, h_emb_flat, e_emb], dim=1) 
        
        return state_vector

    def get_pruned_actions(self, epsilon=10):
        """
        Thực hiện Eq (8): Pruning function g_k((r, e_{k+1}) | u)
        Cắt tỉa không gian hành động, chỉ giữ lại top-epsilon neighbors.
        """
        batch_size = self.current_users.size(0)
        device = self.current_users.device # Lấy device hiện tại
        valid_actions = []
        
        u_emb = self.entity_embs(self.current_users) # (B, dim)
        
        for i in range(batch_size):
            u_id = self.current_users[i].item()
            curr_node = self.current_entities[i].item()
            
            neighbors = self.kg.get_neighbors(curr_node) 
            
            if not neighbors:
                valid_actions.append([]) # Dead end
                continue
                
            # Đã bổ sung `device=device` để tensor tạo ra nằm cùng thiết bị với model
            rels = torch.tensor([n[0] for n in neighbors], device=device)
            next_nodes = torch.tensor([n[1] for n in neighbors], device=device)
            
            r_emb = self.relation_embs(rels)
            next_node_emb = self.entity_embs(next_nodes)
            
            query = u_emb[i] + r_emb 
            scores = torch.sum(query * next_node_emb, dim=1) 
            
            k = min(epsilon, len(scores))
            top_scores, top_indices = torch.topk(scores, k)
            
            actions_i = []
            for idx in top_indices:
                actions_i.append((rels[idx].item(), next_nodes[idx].item())) # Lấy .item() lưu vào list
            valid_actions.append(actions_i)
            
        return valid_actions

    def get_action_space_batch(self):
        """
        [HÀM MỚI] Lấy không gian hành động cho cả Batch và Padding thành Tensor.
        """
        raw_actions_list = self.get_pruned_actions() 
        batch_size = len(raw_actions_list)
        
        lengths = [len(acts) for acts in raw_actions_list]
        max_len = max(lengths) if lengths else 0
        if max_len == 0:
            max_len = 1
            
        device = self.current_entities.device
        
        r_indices = torch.zeros((batch_size, max_len), dtype=torch.long, device=device)
        e_indices = torch.zeros((batch_size, max_len), dtype=torch.long, device=device)
        action_mask = torch.zeros((batch_size, max_len), dtype=torch.float, device=device)
        
        for i, actions in enumerate(raw_actions_list):
            num_acts = len(actions)
            if num_acts > 0:
                rs = [a[0] for a in actions]
                es = [a[1] for a in actions]
                
                r_indices[i, :num_acts] = torch.tensor(rs, device=device)
                e_indices[i, :num_acts] = torch.tensor(es, device=device)
                action_mask[i, :num_acts] = 1.0
                
        r_emb = self.relation_embs(r_indices)
        e_emb = self.entity_embs(e_indices)
        
        action_embs = torch.cat([r_emb, e_emb], dim=-1)
        
        return action_embs, action_mask, raw_actions_list

    def step(self, actions):
        """
        Transition function (Eq 9): Chuyển trạng thái sang bước k+1
        """
        device = self.current_entities.device
        batch_size = len(actions)
        
        rels_list = [a[0] for a in actions]
        ents_list = [a[1] for a in actions]
        
        next_relations = torch.tensor(rels_list, dtype=torch.long, device=device)
        next_entities = torch.tensor(ents_list, dtype=torch.long, device=device)
        
        new_r = next_relations.unsqueeze(1)
        new_e = next_entities.unsqueeze(1)
        new_entry = torch.cat([new_r, new_e], dim=1)
        
        history_shifted = self.path_history[:, 2:]
        self.path_history = torch.cat([history_shifted, new_entry], dim=1)
        
        self.current_entities = next_entities
        self.step_counter += 1
        done = (self.step_counter >= self.max_path_len)
        
        return self._get_state_embedding(), done

    def step_with_indices(self, action_indices, raw_actions_list):
        """
        [HÀM MỚI] Thực hiện bước đi dựa trên Index (0, 1, 2...) mà Agent chọn.
        """
        selected_real_actions = []
        batch_size = len(action_indices)
        
        for i in range(batch_size):
            idx = action_indices[i].item()
            user_acts = raw_actions_list[i]
            
            if len(user_acts) > 0:
                idx = min(idx, len(user_acts) - 1) # Bảo vệ index out of bounds
                real_action = user_acts[idx] 
            else:
                curr_node = self.current_entities[i].item()
                real_action = (0, curr_node) # Dead-end: Tự trỏ về chính nó
                
            selected_real_actions.append(real_action)
            
        return self.step(selected_real_actions)

    def get_reward(self):
        """
        Gọi khi done=True. Tính Time-aware Reward g_R(v|u).
        """
        user_ids = self.current_users
        item_ids = self.current_entities
        
        # Chỉ lấy cột relation từ lịch sử
        history_relation_ids = self.path_history[:, 0::2] 
        
        rewards = self.reward_function(user_ids, item_ids, history_relation_ids)
        return rewards

In [15]:
class TPRecPolicy(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=128, dropout=0.1):
        """
        Mạng Policy (Actor) sử dụng MLP để chấm điểm hành động.
        
        Args:
            state_dim (int): Kích thước vector trạng thái s_k.
                             = dim(u) + dim(flattened_history) + dim(e_k)
            action_dim (int): Kích thước vector hành động a_k.
                              = dim(relation) + dim(next_node)
            hidden_dim (int): Số neuron lớp ẩn.
            dropout (float): Tỷ lệ dropout để tránh overfitting.
        """
        super(TPRecPolicy, self).__init__()
        
        # Layer 1: Kết hợp State và Action
        # Input size = State + Action
        self.fc1 = nn.Linear(state_dim + action_dim, hidden_dim)
        
        # Activation function (ELU hoặc ReLU thường dùng trong Graph RL)
        self.act = nn.ELU()
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
        # Layer 2: Output ra 1 điểm số (Scalar Score) cho cặp (State, Action)
        self.fc2 = nn.Linear(hidden_dim, 1)

    def forward(self, state_emb, action_embs, action_mask=None):
        """
        Forward pass để tính xác suất hành động.
        
        Args:
            state_emb: (Batch, State_Dim) - Vector trạng thái s_k
            action_embs: (Batch, Max_Actions, Action_Dim) - Các hành động ứng viên
            action_mask: (Batch, Max_Actions) - Mask (1=Hợp lệ, 0=Padding)
            
        Returns:
            probs: (Batch, Max_Actions) - Xác suất (đã qua Softmax)
            log_probs: (Batch, Max_Actions) - Log xác suất (để tính Loss)
        """
        batch_size, num_actions, _ = action_embs.size()
        
        # 1. Mở rộng State để khớp với số lượng Action (Broadcasting)
        # state: (B, S_Dim) -> (B, 1, S_Dim) -> (B, K, S_Dim)
        state_expanded = state_emb.unsqueeze(1).expand(-1, num_actions, -1)
        
        # 2. Ghép (Concatenate) State và Action
        # input: (B, K, S_Dim + A_Dim)
        inputs = torch.cat([state_expanded, action_embs], dim=2)
        
        # 3. Qua mạng MLP
        x = self.fc1(inputs)
        x = self.act(x)
        x = self.dropout(x)
        scores = self.fc2(x).squeeze(-1) # (B, K, 1) -> (B, K)
        
        # 4. Masking (Cực kỳ quan trọng)
        # Gán điểm số rất thấp (-1e9) cho các hành động padding để Softmax = 0
        if action_mask is not None:
            scores = scores.masked_fill(action_mask == 0, -1e9)
        
        # 5. Tính xác suất (Softmax)
        probs = F.softmax(scores, dim=1)
        
        # Trả về cả log_probs để tiện tính REINFORCE Loss sau này
        log_probs = F.log_softmax(scores, dim=1)
        
        return probs, log_probs

### Class Config 

In [16]:
class Config:
    # Đường dẫn dữ liệu
    TCKG_PATH = f"./data/{name}/{name}_TCKG.csv"  # File CSV chứa đồ thị (như bạn đã gửi)
    # TRAIN_USERS_PATH = "data/train_users.csv" # File chứa danh sách user ID dùng để train
    SAVE_DIR = "./checkpoints"
    
    # Siêu tham số Model
    EMBED_DIM = 64
    HIDDEN_DIM = 256
    HISTORY_LEN = 3   # k' (độ dài lịch sử)
    MAX_PATH_LEN = 3  # K (số bước đi)
    
    # Siêu tham số Training
    BATCH_SIZE = 512 # Batch lớn giúp RL ổn định hơn
    NUM_EPOCHS = 500
    LEARNING_RATE = 5e-4
    BETA_ENTROPY = 0.01
    
    # Thiết bị
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # IDs của các Relation Interaction (Quan trọng cho Reward)
    # Ví dụ: 20=interacted_0, 21=interacted_1, 22=interacted_2
    INTERACTION_CLUSTER_IDS = [20, 21, 22, 23] 

### Load Dataset

In [17]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

# 1. Tạo class Dataset làm cầu nối
class InteractionDataset(Dataset):
    def __init__(self, df):
        """Nhận vào một DataFrame và chuyển đổi các cột cần thiết thành mảng NumPy"""
        self.users = df['user_id'].values
        self.entities = df['entity_id'].values

    def __len__(self):
        """Trả về tổng số dòng dữ liệu"""
        return len(self.users)

    def __getitem__(self, idx):
        """Lấy dữ liệu tại vị trí idx và biến thành PyTorch Tensor"""
        user_tensor = torch.tensor(self.users[idx], dtype=torch.long)
        entity_tensor = torch.tensor(self.entities[idx], dtype=torch.long)
        
        return user_tensor

# 2. Đọc file CSV thành Pandas DataFrame
# (Lưu ý nhỏ: Mình giữ nguyên tên file của bạn, nhưng chữ 'interacions' hình như đang thiếu chữ 't', bạn nhớ kiểm tra lại tên file thật nhé)
train_df = pd.read_csv(f'./data/{name}/{name}_train_interactions.csv')
val_df = pd.read_csv(f'./data/{name}/{name}_val_interactions.csv')
test_df = pd.read_csv(f'./data/{name}/{name}_test_interactions.csv')

# 3. Bọc DataFrame vào class Dataset vừa tạo
train_dataset = InteractionDataset(train_df)
val_dataset = InteractionDataset(val_df)
test_dataset = InteractionDataset(test_df)

# 4. Đưa Dataset vào DataLoader
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False, num_workers=0)

### Main function

In [18]:
def train(env, policy_net, train_loader, num_epochs=50, 
          learning_rate=1e-3, beta_entropy=0.01, 
          save_path="./checkpoints", device="cuda"):
    """
    Hàm huấn luyện gộp (không dùng hàm phụ trợ train_one_epoch).
    
    Cấu trúc lồng nhau:
    1. Loop Epochs (Hàng trăm lần)
       2. Loop Batches (Duyệt qua toàn bộ dataset)
          3. Loop Steps (Agent đi k bước tìm đường)
             -> Tính Reward & Loss
             -> Backpropagation
    """
    
    # --- 1. SETUP ---
    if not os.path.exists(save_path):
        os.makedirs(save_path)
        
    # Đưa model và env vào thiết bị (GPU/CPU)
    policy_net.to(device)
    # env.to(device) # Nếu class Env của bạn có hỗ trợ .to()
    
    # Optimizer (Adam)
    optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate)
    
    # Learning Rate Scheduler (Giảm LR sau mỗi 50 epoch)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.9)
    
    print(f"🚀 Bắt đầu huấn luyện {num_epochs} epochs...")
    start_time = time.time()

    print(f"Tổng số mẫu trong Dataset: {len(train_loader.dataset)}")
    print(f"Tổng số Batch sẽ chạy: {len(train_loader)}")
    
    # --- 2. VÒNG LẶP EPOCH (Vòng ngoài cùng) ---
    for epoch in range(1, num_epochs + 1):
        
        policy_net.train() # Bật chế độ train (Dropout, Batchnorm...)
        
        total_epoch_loss = 0
        total_epoch_reward = 0
        num_batches = 0
        
        # Thanh tiến trình cho từng Epoch
        pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")
        
        # --- 3. VÒNG LẶP BATCH (Duyệt Dataset) ---
        for batch_users in train_loader:
            batch_users = batch_users.to(device)
            
            # A. Reset môi trường cho batch mới
            # state_emb: (Batch, State_Dim)
            state_emb = env.reset(batch_users)
            
            # Danh sách lưu lại thông tin để tính Loss sau khi đi xong
            saved_log_probs = [] # Log π(a|s)
            saved_entropies = [] # H(π)
            
            # --- 4. VÒNG LẶP STEP (Agent đi tìm đường) ---
            # Đi max_path_len bước (ví dụ: 3 bước)
            for t in range(env.max_path_len):
                
                # B. Lấy hành động khả thi (Pruned Action Space)
                # action_embs: (Batch, K, Dim)
                # mask: (Batch, K) - Để che padding
                # raw_actions: List để map lại ID thực tế
                action_embs, action_mask, raw_actions = env.get_action_space_batch()
                
                # C. Policy Forward (Ra quyết định)
                probs, log_probs = policy_net(state_emb, action_embs, action_mask)
                
                # D. Chọn hành động (Sampling dựa trên xác suất)
                m = torch.distributions.Categorical(probs)
                action_indices = m.sample() # (Batch,)
                
                # E. Lưu Log Prob và Entropy
                saved_log_probs.append(m.log_prob(action_indices))
                saved_entropies.append(m.entropy())
                
                # F. Thực hiện bước đi (Transition)
                # Hàm này cập nhật vị trí agent và lịch sử
                state_emb, done = env.step_with_indices(action_indices, raw_actions)
            
            # --- 5. TÍNH TOÁN LOSS & UPDATE (Kết thúc 1 trajectory) ---
            
            # G. Tính Reward cuối cùng (Terminal Reward)
            # rewards shape: (Batch,)
            rewards = env.get_reward() 
            
            # H. Tính REINFORCE Loss
            # Loss = - sum(log_prob * reward) - beta * entropy
            batch_loss = 0
            
            # Cộng dồn loss qua các bước thời gian (t=1..K)
            # Giả sử Reward ở bước cuối áp dụng cho toàn bộ chuỗi hành động
            for log_prob in saved_log_probs:
                # Dấu trừ (-) để chuyển bài toán Maximize Reward thành Minimize Loss
                batch_loss += -log_prob * rewards
                
            # Trừ đi Entropy (Khuyến khích khám phá)
            for entropy in saved_entropies:
                batch_loss -= beta_entropy * entropy
                
            # Lấy trung bình loss trên toàn bộ Batch users
            loss = batch_loss.mean()
            
            # I. Backpropagation (Lan truyền ngược)
            optimizer.zero_grad() # Xóa gradient cũ
            loss.backward()       # Tính gradient mới
            
            # Kỹ thuật Gradient Clipping (Tránh bùng nổ gradient - Rất quan trọng trong RL)
            torch.nn.utils.clip_grad_norm_(policy_net.parameters(), max_norm=1.0)
            
            optimizer.step()      # Cập nhật trọng số
            
            # --- 6. LOGGING ---
            total_epoch_loss += loss.item()
            total_epoch_reward += rewards.mean().item()
            num_batches += 1
            
            # Cập nhật thanh tiến trình (Hiển thị loss/reward hiện tại)
            pbar.set_postfix({'Loss': f"{loss.item():.4f}", 'R': f"{rewards.mean().item():.4f}"})
            
        # --- KẾT THÚC 1 EPOCH ---
        scheduler.step() # Cập nhật Learning Rate
        
        avg_loss = total_epoch_loss / num_batches
        avg_reward = total_epoch_reward / num_batches
        
        # In kết quả Epoch
        # print(f"\n[Kết thúc Epoch {epoch}] Avg Loss: {avg_loss:.4f} | Avg Reward: {avg_reward:.4f}")
        
        # Lưu Checkpoint mỗi 10 Epoch
        if epoch % 10 == 0:
            ckpt_path = os.path.join(save_path, f"model_epoch_{epoch}.pt")
            torch.save({
                'epoch': epoch,
                'model_state_dict': policy_net.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_loss,
            }, ckpt_path)
            # print(f"💾 Đã lưu model tại {ckpt_path}")

    print(f"✅ Hoàn thành training sau {(time.time() - start_time):.0f} giây.")

def main():
    cfg = Config()
    print(f"🔥 Starting TPRec Training on {cfg.DEVICE}...")
    
    # -------------------------------------------------
    # BƯỚC 1: Xây dựng Đồ thị Tri thức (TCKG)
    # -------------------------------------------------
    # Lưu ý: Cần đảm bảo file CSV đã được map ID về dạng số nguyên liên tục (0, 1, 2...)
    print("Loading Knowledge Graph...")
    # Tính offset tự động (như class TCKG tối ưu tôi đã viết)
    tckg = TCKG(cfg.TCKG_PATH) 

    # -------------------------------------------------
    # BƯỚC 2: Khởi tạo Embeddings từ file Pickle
    # -------------------------------------------------
    print("Loading Pre-trained TransE Embeddings...")
    pickle_file_path = './pickle/book_transE_embeddings_2026-02-19_14-30-57.pkl'

    with open(pickle_file_path, 'rb') as f:
        saved_data = pickle.load(f)
        
    pretrained_ent = saved_data['entity_embeddings']
    pretrained_rel = saved_data['relation_embeddings']
    
    # 2. Chuyển đổi sang PyTorch Tensor (ép kiểu Float32 để tính toán neural network)
    # Nếu data đang là Numpy array:
    if isinstance(pretrained_ent, np.ndarray):
        ent_tensor = torch.tensor(pretrained_ent, dtype=torch.float32)
        rel_tensor = torch.tensor(pretrained_rel, dtype=torch.float32)
    else:
        # Nếu data đã là Tensor sẵn:
        ent_tensor = pretrained_ent.clone().detach().float()
        rel_tensor = pretrained_rel.clone().detach().float()
        
    # 3. Nạp vào nn.Embedding
    # freeze=False: Cho phép RL Agent tiếp tục cập nhật (fine-tune) vector trong lúc tìm đường
    # freeze=True: Khóa cứng vector, RL Agent chỉ học Policy Network (Học nhanh hơn, chống overfit)
    entity_embs = nn.Embedding.from_pretrained(ent_tensor, freeze=False)
    relation_embs = nn.Embedding.from_pretrained(rel_tensor, freeze=False)

    
    # -------------------------------------------------
    # BƯỚC 3: Khởi tạo Reward Function
    # -------------------------------------------------
    print("Setting up Reward Function...")
    reward_func = TimeAwareRewardFunction(
        user_embs=entity_embs,    # Chia sẻ trọng số với Env
        entity_embs=entity_embs,
        relation_embs=relation_embs,
        interaction_cluster_ids=cfg.INTERACTION_CLUSTER_IDS,
        bias_embs=None, # Tự tạo bias mới
        temperature= 2.0 
    )
    
    # -------------------------------------------------
    # BƯỚC 4: Khởi tạo Môi trường (Environment)
    # -------------------------------------------------
    print("Setting up Environment...")
    env = TPRecEnvironment(
        kg=tckg,
        entity_embeddings=entity_embs,
        relation_embeddings=relation_embs,
        reward_function=reward_func, # Inject reward vào env
        max_path_len=cfg.MAX_PATH_LEN,
        history_len=cfg.HISTORY_LEN
    )
    
    # -------------------------------------------------
    # BƯỚC 5: Khởi tạo Policy Network (Agent)
    # -------------------------------------------------
    print("Building Policy Network...")
    
    # Tính toán kích thước State và Action theo công thức chuẩn
    # Action = Relation + Entity
    action_dim = cfg.EMBED_DIM + cfg.EMBED_DIM 
    
    # State = User + Flattened_History + Current_Entity
    # Flattened_History = k' * (Relation + Entity)
    history_flat_dim = cfg.HISTORY_LEN * (cfg.EMBED_DIM + cfg.EMBED_DIM)
    state_dim = cfg.EMBED_DIM + history_flat_dim + cfg.EMBED_DIM
    
    print(f"--> State Dim: {state_dim} | Action Dim: {action_dim}")
    
    policy_net = TPRecPolicy(
        state_dim=state_dim,
        action_dim=action_dim,
        hidden_dim=cfg.HIDDEN_DIM
    )
    
    
    # -------------------------------------------------
    # BƯỚC 7: BẮT ĐẦU TRAINING
    # -------------------------------------------------
    # Gọi hàm train monolithic (tất cả trong một) bạn đã chọn

    train(
        env=env,
        policy_net=policy_net,
        train_loader=train_loader,
        num_epochs=cfg.NUM_EPOCHS,
        learning_rate=cfg.LEARNING_RATE,
        beta_entropy=cfg.BETA_ENTROPY,
        save_path=cfg.SAVE_DIR,
        device=cfg.DEVICE
    )

if __name__ == "__main__":
    # Tạo thư mục data giả lập nếu chưa có để test logic (Tùy chọn)
    if not os.path.exists("checkpoints"):
        os.makedirs("checkpoints")
        
    main()

🔥 Starting TPRec Training on cpu...
Loading Knowledge Graph...
Loading TCKG from ./data/book/book_TCKG.csv...
Inverse Relation Offset: 24. Total Relation Space: 49
TCKG Loaded successfully. Graph construction complete.
Loading Pre-trained TransE Embeddings...
Setting up Reward Function...
Setting up Environment...
Building Policy Network...
--> State Dim: 512 | Action Dim: 128
🚀 Bắt đầu huấn luyện 500 epochs...
Tổng số mẫu trong Dataset: 14683
Tổng số Batch sẽ chạy: 15


Epoch 1/500:   0%|          | 0/15 [00:10<?, ?it/s, Loss=3.3544, R=0.5030]















Epoch 3/500:   0%|          | 0/15 [00:08<?, ?it/s, Loss=3.3806, R=0.5032]

KeyboardInterrupt: 